### Import Dependencies

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

### Instantiate Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### Encode and Calculate Sentiment

In [7]:
tokens  = tokenizer.encode("I enjoy studying Machine Learning as it is quite intriguing", return_tensors='pt')
result = model(tokens)

In [8]:
int(torch.argmax(result.logits))+1

4

### Collect Reviews

In [26]:
r = requests.get('https://www.yelp.com/biz/star-india-restaurant-san-francisco-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [27]:
reviews

['Food: 10/10 has all the traditional options including vegetarian options! chicken tikka masala is a must. also have drinks like chai & manService: 10/10 super friendly, nice lady let us walk in and check out the buffet (offered from 11am-4pm weekends and weekdays) when deciding where to eat. also ordered us naan for the tableAmbiance: 10/10 nice Indian music playing in the background, decor is inviting',
 "Food was super yummy and authentic!  Service was incredible! The waitress is super nice!! Chai was a must have!! ambiance, so pretty and I feel like I'm at home!! (::",
 'Good and comfort Indian food, young lady is very attentive, wonderful customer service! Cozy ambiance!',
 'More of a 4.5, but that\'s still pretty good! I ended up coming to Star India after watching enough Ted Lasso to want to find something even remotely good as the Indian food in London. Initially, I had been seeking another spot for their Butter Chicken, but was utterly disappointed when the other place I had 

### Load Reviews into DataFrame and Score

In [28]:
import numpy as np
import pandas as pd

In [29]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [30]:
df.head()

,review
0,Food: 10/10 has all the traditional options in...
1,Food was super yummy and authentic! Service w...
2,"Good and comfort Indian food, young lady is ve..."
3,"More of a 4.5, but that's still pretty good! I..."
4,We came in around 8pm on Friday and to our sur...


In [31]:
example = df['review'].iloc[0]

In [32]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [33]:
sentiment_score(example)

5

In [34]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [35]:
df

,review,sentiment
0,Food: 10/10 has all the traditional options in...,5
1,Food was super yummy and authentic! Service w...,5
2,"Good and comfort Indian food, young lady is ve...",5
3,"More of a 4.5, but that's still pretty good! I...",4
4,We came in around 8pm on Friday and to our sur...,1
5,I'm surprised that this place exists in SF dow...,3
6,Star India is the OG buffet of the 90's... I r...,5
7,While the food is tasty ... the price vs porti...,3
8,This is a really nice India resturant. Great c...,5
9,I recently spent the day in the city with my M...,4
